In [ ]:
import torch

# torch版本
torch.__version__ # '1.7.0'
'''
1. 张量的创建
2. 张量信息统计
3. 在张量上进行操作
4. 张量运算
5. CPU/GPU相关
6. 存储和下载
'''

In [ ]:
# 张量的创建

# (1): 随机创建
# torch.randint(low=0, high, size) 可重复; [low, high)
torch.randint(0, 8, (5,), dtype=torch.int64) # tensor([7, 2, 0, 5, 0]) 

torch.rand(2, 5) 
# torch.rand((2, 5))也可  
# 得到[0, 1)均匀分布
input = torch.tensor([1,2,3,4]).float() 
torch.rand_like(input) # 等价于torch.rand(input_size)
torch.rand(4)

torch.randn # 得到标准正态分布

a = torch.empty(5,2) # 随机生成, 值是随机的
torch.empty(0) # tensor([])
b = torch.Tensor(2,3) # 随机生成, 值是随机的

a.random_(10) # 随机生成0-9, 浮点
a.uniform_(0, 1)

# 伯努利分布
prob = torch.empty(3, 3).uniform_(0, 1)
torch.bernoulli(prob) # 输出形状与prob一致
# 第i个元素的输出tensor将要根据输入的第i个概率值来决定是否生成1值
# tensor.bernoulli_(prob)

# torch.distributions
# OneHotCategorical: 创建一个one-hot类分布
        # 可传入参数probs和logits
        # 其中probs: event probabilities, logits: event log probabilities
m = torch.distributions.OneHotCategorical(torch.tensor([0.25, 0.25, 0.25, 0.25])) # 参数为probs
m.sample() # 样本为one-hot编码向量, size为probs.size(-1), 如tensor([0., 1., 0., 0.])
torch.distributions.OneHotCategorical(logits=torch.zeros(8, dtype = torch.float32)).sample([10]) # 10x8

# Normal: 参数为loc和scale
n = torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([1.0]))
n.sample()  # normally distributed with loc=0 and scale=1, 如tensor([0.9128])
means = torch.tensor([[2., 0.], [0., 2.], [-2., 0.], [0., -2.]])
samples = torch.distributions.normal.Normal(loc=means, scale=0.1).sample() # 4x2

# (2): 指定
a = torch.arange(5)[:, None] # 5x1 0-4 'torch.LongTensor' 等价于a = a.unsqueeze(1)
b = torch.arange(2)[None, :] # 1x2 0-1
# a-b为 5x2
torch.arange(3., 8 + 1) # 'torch.FloatTensor' [3., 8.]
torch.arange(5.7, -2.1, -3) # tensor([ 5.7000,  2.7000, -0.3000])

torch.linspace(3, 8, 5) # 'torch.FloatTensor' tensor([3.0000, 4.2500, 5.5000, 6.7500, 8.0000])
# 包含首尾值

torch.zeros(3, 5)
# 可以方便的复制原来tensor的所有类型, 比如数据类型和数据所在设备等等
a.new_zeros(2, 1)
torch.zeros_like(a), torch.zeros(a.size())
torch.ones(3, 3) # torch.ones((3, 3))也可, torch.FloatTensor

torch.isnan()
torch.reciprocal # 倒数

torch.eye(3) # torch.FloatTensor
'''
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
'''

torch.LongTensor([[0, 1, 2, 0, 0], [2, 0, 0, 1, 2]])
torch.tensor([-0.3623, -0.6115,  0.7283,  0.4699,  2.3261,  0.1599]) # 'torch.FloatTensor'

torch.tensor([1,2,3]) # 'torch.LongTensor'
torch.Tensor([1,2,3]) # 'torch.FloatTensor'

torch.tensor(0) # tensor(0)
torch.Tensor(1) # tensor([0.])

image = torch.zeros_like(a)
image.copy_(a)

# (3): 从numpy创建
import numpy as np
n_np = np.arange(5)
n = torch.from_numpy(n_np) # 共享内存
n = torch.from_numpy(n_np.copy())  # If ndarray has negative stride, 不共享内存
# negative stride表示np.ndarray执行了torch.Tensor没有的方法就会报错, 使用.copy()创建副本之后二者相互独立
# 或
n = torch.tensor(n_np) # torch.Tensor(n_np)

# 把tensor转换成numpy的格式(array)
n.cpu().data.numpy()
# 或
n.data.cpu().numpy()  # .data是获取tensor; .data返回的是一个新的Tensor对象, 因此它们的id不同, 说明二者不是同一个Tensor; 但它们共享数据的存储空间, 即二者的数据部分指向同一块内存, 因此修改b的元素时, a的元素也对应修改

# 除了CharTensor, 其他所有CPU上的张量都支持转换为numpy格式然后再转换回来

In [ ]:
# 张量信息统计

# (1): 统计数目
tensor = torch.zeros(3, 5)
tensor.dim(), tensor.size(), tensor.shape # 2, torch.Size([3, 5]), torch.Size([3, 5])
# .size(): tuple的一个子类, 等价于.shape
tensor.numel() # 15

# 统计 tensor 的元素个数
input = torch.tensor([1,2,3,4])
input.nelement(), input.numel() # 4, 4

# torch.bincount: 计算非负数组中每个值的频率, 值对应返回数组的索引, 频率对应返回数组的元素值
input = torch.randint(0, 8, (5,), dtype=torch.int64) # tensor([4, 2, 0, 2, 6])
torch.bincount(input) # tensor([1, 0, 2, 0, 1, 0, 1])

torch.mean(input, dim=0)

# 得到非零/零元素
t = torch.tensor([0,1,2,1])
print(torch.nonzero(t)) # 得到非零元素索引
print(torch.nonzero(t == 0)) # 得到零元素索引
'''
tensor([[1],
        [2],
        [3]])
tensor([[0]])
'''

# (2): 数据类型
# 设置默认数据类型
torch.set_default_tensor_type(torch.FloatTensor)
# 在PyTorch中, float比double更快

a = torch.tensor([0.1, 0.2])
b = torch.FloatTensor([0.1, 0.2])
c = torch.LongTensor([0, 1, 2])
a.type(), b.type(), c.type(), type(a), type(b), type(c), a.dtype, b.dtype, c.dtype
# .type(tensor.dtype)
# 'torch.FloatTensor', 'torch.FloatTensor', 'torch.LongTensor', torch.Tensor, torch.Tensor, torch.Tensor, torch.float32, torch.float32, torch.int64
a = a.double() # 'torch.DoubleTensor', 等价于a = a.type(torch.DoubleTensor)
torch.tensor([1,2,3,4]) # 'torch.LongTensor'
torch.tensor([1,2,3,4]).float() # 'torch.FloatTensor'
# .long(); .byte()

# torch.ByteTensor, torch.cuda.ByteTensor
# torch.CharTensor, torch.cuda.CharTensor
# torch.ShortTensor, torch.cuda.ShortTensor
# torch.IntTensor, torch.cuda.IntTensor
# torch.LongTensor, torch.cuda.LongTensor
# torch.FloatTensor, torch.cuda.FloatTensor
# torch.DoubleTensor, torch.cuda.DoubleTensor
# torch.HalfTensor, torch.cuda.HalfTensor
# torch.BoolTensor, torch.cuda.BoolTensor

# dtype
# torch.uint8, torch.int8, torch.int16/torch.short, torch.int32/torch.int, torch.int64/torch.long
# torch.float32/torch.float, torch.float64/torch.double, torch.float16/torch.half
# torch.bool

# (3): 梯度
# tensor.requires_grad默认为False, 因此.grad没有输出
# requires_grad为True时, 表示需要计算Tensor的梯度
# requires_grad=False可以用来冻结部分网络, 只更新另一部分网络的参数
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)
# 构建计算图
y = w * x + b    # y = 2 * x + 3
# 计算梯度
y.backward()
print(x.grad)    
print(w.grad)   
print(b.grad)  
'''
tensor(2.)
tensor(1.)
tensor(1.)
'''  
x = torch.tensor([[1, 2], [3, 4]], requires_grad=True, dtype=torch.float32)
print(x)
'''
tensor([[1., 2.],
        [3., 4.]], requires_grad=True)
'''
y = x - 2
print(y) 
'''
tensor([[-1.,  0.],
        [ 1.,  2.]], grad_fn=<SubBackward0>)
'''
print(y.grad_fn) # 因为y是来自一次操作，所以它有grad_fn; <SubBackward0 object at 0x7fc896771c90>
print(x.grad_fn) # None
print(y.grad_fn.next_functions) # ((<AccumulateGrad object at 0x7fc8967dddd0>, 0), (None, 0))
print(y.grad_fn.next_functions[0][0]) # <AccumulateGrad object at 0x7fc8967ddf10>
# 计算图终止于叶子AccumulateGrad节点, 有一个.variable属性指向叶子节点
print(y.grad_fn.next_functions[0][0].variable)
'''
tensor([[1., 2.],
        [3., 4.]], requires_grad=True)
'''
z = y * y * 3
a = z.mean() 
print(z)
'''
tensor([[ 3.,  0.],
        [ 3., 12.]], grad_fn=<MulBackward0>)
'''
print(a)
'''
tensor(4.5000, grad_fn=<MeanBackward0>)
'''
a.backward() # 计算梯度da/dx
print(x.grad)
'''
tensor([[-1.5000,  0.0000],
        [ 1.5000,  3.0000]])
'''

# tensor.requires_grad_()函数会改变Tensor的requires_grad属性并返回Tensor, 如返回tensor([1., 2.], requires_grad=True)
# 修改requires_grad的操作是原位操作(in place)
# 该函数的默认参数为requires_grad=True
	# requires_grad=True时, 自动求导会记录对Tensor的操作
	# requires_grad_()的主要用途是告诉自动求导开始记录对Tensor的操作
# 对上述tensor进行运算并执行.backward(); 上述tensor的.grad有输出

# detach()函数会返回一个新的Tensor对象b, 并且新Tensor是与当前的计算图分离的
# 其requires_grad属性为False, 反向传播时不会计算其梯度
# b与a共享数据的存储空间, 二者指向同一块内存
# 共享内存空间只是共享的数据部分, a.grad与b.grad是不同的

# .retain_grad()
# 对非叶节点张量(即中间节点张量)启用, 用于保存梯度的属性(.grad)
# 默认情况下, 对于非叶节点张量是禁用属性.grad, 计算完梯度之后就被释放回收内存, 不会保存中间结果的梯度

# torch.no_grad()是一个上下文管理器, 用来禁止梯度的计算, 通常用在网络推断中, 它可以减少计算内存的使用量

torch.autograd.set_detect_anomaly(True) # debug
# 为了更快找到可能的错误, 会减慢训练

In [ ]:
# 在张量上进行操作

# (1): 复制
a = torch.arange(5)[:, None] # 5x1
b = a.repeat(1, 2) # 5x2 在最后一个维度进行复制
# repeat拷贝张量的数据, 参数是各个维度上重复的次数
# expand张量不会分配新的内存, 只是在存在的张量上创建一个新的视图(view); 参数是想要得到的最后张量的形状; expand_as()    
t = torch.tensor([[0,1,2,1], [2,4,1,4]])
a = torch.reshape(t, (4, 2, 1)).expand(4, 2, 2)
'''
tensor([[[0, 0],
         [1, 1]],

        [[2, 2],
         [1, 1]],

        [[2, 2],
         [4, 4]],

        [[1, 1],
         [4, 4]]])
'''

# 有三种复制的方式, 对应不同的需求:
        # 1. tensor.clone(): 新内存; 仍在计算图中
        # 2. tensor.detach(): 共享内存; 不在计算图中
        # 3. tensor.detach().clone(): 新内存; 不在计算图中

# (2): 切片(索引)
# select(dim, index)
b.select(0, 0) # tensor([0, 0]) 结果张量dim那一维消失

# 从矩阵中获取值
'''
m = tensor([[2., 5., 3., 7.],
        [4., 2., 1., 9.]])
'''
tensor[0][2] # tensor(3.)
tensor[0, 2] # tensor(3.)
tensor[:, 1] # tensor([5., 2.]), 降维
tensor[:, [1]] # tensor([[5.], [2.]]), 不降维, 2x1
tensor[[0], :] # tensor([[2., 5., 3., 7.]]), 不降维, 1x4
tensor[0, :] # tensor([2., 5., 3., 7.]), 降维

example = torch.Tensor([[2,3], [1, 3]])
example[...,0] # tensor([2., 1.])

# tensor = tensor[torch.randperm(tensor.size(0))]  # shuffle第一维

# pytorch不支持tensor[::-1]这样的负步长操作, 水平翻转可以通过张量索引实现
example = torch.Tensor([[2,3], [1, 3]])
print(example)
example[:, torch.arange(example.size(1) - 1, -1, -1).long()] # 等价于torch.flip(example, [1])
'''
tensor([[2., 3.],
        [1., 3.]])
tensor([[3., 2.],
        [3., 1.]])
'''

# (3): 张量命名
# 在pytorch1.3之前, 需要使用注释
# Tensor[N, C, H, W]
images = torch.randn(32, 3, 56, 56)
images.sum(dim=1)
images.select(dim=1, index=0)
# pytorch1.3之后
NCHW = ['N', 'C', 'H', 'W']
images = torch.randn(32, 3, 56, 56, names=NCHW)
images.sum('C')
images.select('C', index=0)
# 也可以这么设置
tensor = torch.rand(3,4,1,2,names=('C', 'N', 'H', 'W'))

# (4): 排序(维度变换)
# 使用align_to可以对维度方便地排序(维度交换)
tensor = tensor.align_to('N', 'C', 'H', 'W')
tensor.permute(2,0,1)

# 矩阵转置
tensor.t()
tensor.transpose(0, 1)
# 在使用transpose()进行转置操作时, pytorch并不会创建新的, 转置后的tensor
# 而是修改了tensor中的一些属性(也就是元数据), 使得此时的offset和stride是与转置tensor相对应的
# 转置的tensor和原tensor的内存是共享的
# 转置前的是contiguous, 转置后的不是   
# 经过转置后得到的tensor, 它内部数据的布局方式和从头开始创建一个常规的tensor的布局方式是不一样的    
# 当调用contiguous()时, 会强制拷贝一份tensor, 让它的布局和从头创建的一模一样
# permute()和tranpose()比较相似, transpose是交换两个维度, permute()是交换多个维度

# transpose = (lambda b: b.t_().squeeze(0).contiguous())  
# transpose(text_batch)和text_batch=transpose(text_batch)得到的结果不同
a = torch.Tensor([[1,2],[2,2],[3,2]])
transpose = (lambda b: b.t_().squeeze(0).contiguous()) 
b = transpose(a)
print(transpose(a)) # 保持原始张量
print(b)
'''
tensor([[1., 2.],
        [2., 2.],
        [3., 2.]])
tensor([[1., 2., 3.],
        [2., 2., 2.]])
'''

a = torch.Tensor([[1,2],[2,2],[3,2]])
torch.flatten(a, 0) # tensor([1., 2., 2., 2., 3., 2.])

torch.sort(tensor, dim=1, descending=False) 
# 默认dim=1(按行排序)
# descending=True: 从大到小排序, descending=False: 从小到大排序; 默认descending=False
# 输出value, index
# torch.argsort只输出index

# torch.roll(input, shifts, dims)
# shifts: 变换的幅度, 为整数或者元组; 若为元组, 其shape与dims保持一样
x = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9]).view(3, 3)
print(x)
torch.roll(x, (1 , 1), (0 , 0))
torch.roll(x, (-1 , -1), (0 , 0))
torch.roll(x, (-1 , -1), (0 , 1))
'''
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
tensor([[4, 5, 6],
        [7, 8, 9],
        [1, 2, 3]])
tensor([[7, 8, 9],
        [1, 2, 3],
        [4, 5, 6]])
tensor([[5, 6, 4],
        [8, 9, 7],
        [2, 3, 1]])
'''

# (5): 指定值
c = torch.empty(5,2,6) # 随机生成
c.zero_() # 全部置零

m = torch.Tensor([[2, 5, 3, 7],[4, 2, 1, 9]])
m.sub_(1)
'''
tensor([[1., 4., 2., 6.],
        [3., 1., 0., 8.]])
'''

example = torch.Tensor(2, 3, 4)
example.random_(10) # 0-9
r = torch.Tensor(example)
r.resize_(2, 4, 3)
r.zero_() 
# example值全变为0, size不变
print(example)
'''
tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])
'''
s = r.clone()
s.fill_(1) # 值全为1
print(s)
print(r) # 值不受影响
'''
tensor([[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]])
tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])
'''

mask = torch.Tensor([[1,0],[0,0],[1,1]])
mask.masked_fill(mask == 1, value=3) # 用value填充mask中值为1位置相对应的元素
'''
tensor([[3., 0.],
        [0., 0.],
        [3., 3.]])
'''

# 指定值的区间
f = torch.tensor([[1,2,0], [3,0,0], [4,5,6]])
torch.clamp(f, 1, 3) # 将输入张量每个元素的夹紧到区间[1,3]
'''
tensor([[1, 2, 1],
         [3, 1, 1],
         [3, 3, 3]])
'''

# 放置元素
torch.zeros(3, 5).scatter_(0, torch.LongTensor([[0, 1, 2, 0, 0], [2, 0, 0, 1, 2]]), 1)
'''
tensor([[1., 1., 1., 1., 1.],
        [0., 1., 0., 1., 0.],
        [1., 0., 1., 0., 1.]])
'''
# (dim, index, src) 如何放置按照参数1和参数2的规定, 放置哪些元素, 按照参数3的规定
# index和src的shape一样 
# 先看index的元素值, 结合dim和index元素值的位置确定放置位置, 放置元素src所在对应index元素的位置
# 按行填充(dim=0), 被填充tensor的行数与index中的最大值相同(index中的元素值指明填充第几行, 第几列由index中的元素所在列确定), 被填充tensor的列数与src的列数相同
# 将整数标记转换成独热(one-hot)编码
t = torch.tensor([0,1,2,1])
N = t.size(0)
num_classes = 3
one_hot = torch.zeros(N, num_classes).long()
one_hot.scatter_(dim=1, index=torch.unsqueeze(t, dim=1), src=torch.ones(N, num_classes).long()) 
# 将src中数据根据index中的索引按照dim的方向填进input中, index为src中的每一个数据指明在input中的dim方向上的索引, index, src和input的另外一个dim的大小要一致
print(one_hot)
'''
tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 1, 0]])
'''

# (6): 取值
# 从只包含一个元素的张量中提取值
value = torch.rand(1).item()
# torch.rand(1) --> tensor([0.4218])

# torch.gather(input, dim, index) 
# 输出和index的维度是一致的
input = [
    [2, 3, 4, 5, 0, 0],
    [1, 4, 3, 0, 0, 0],
    [4, 2, 2, 5, 7, 0],
    [1, 0, 0, 0, 0, 0]
]
input = torch.tensor(input)
# 注意index的类型
length = torch.LongTensor([[3],[2],[4],[0]])
torch.gather(input, 1, length)
'''
tensor([[5],
        [3],
        [7],
        [1]])
'''

x = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(torch.narrow(x, 0, 0, 2)) # 第二个参数是dim, 第三个参数是起始dim, 第四个参数是长度
# tensor([[ 1,  2,  3], [ 4,  5,  6]])
print(torch.narrow(x, 1, 1, 2))
# tensor([[ 2,  3], [ 5,  6], [ 8,  9]])

# (7): 形变
# torch.view
tensor.view(2, 3)
# tensor.view(-1) --> 原张量会变成一维
# tensor.view(2, -1) --> 自动补齐列向量长度
# 与torch.view相比, torch.reshape可以自动处理输入张量不连续的情况
tensor = torch.rand(2,3,4)
shape = (6, 4)
tensor = torch.reshape(tensor, shape)

tensor.resize_(2,4,3)

# torch.tril(inputs, diagonal) 返回inputs的下三角部分, diagonal=0: 主对角线及其下部分被保留, -1则不保留主对角线上的元素
a = torch.randn(3, 3)
print(a)
print(torch.tril(a))
print(torch.tril(a, -1))
'''
tensor([[-0.0720, -0.9020,  0.3481],
        [-0.6275, -0.5469,  0.0448],
        [-0.1785,  1.7177, -0.5258]])
tensor([[-0.0720,  0.0000,  0.0000],
        [-0.6275, -0.5469,  0.0000],
        [-0.1785,  1.7177, -0.5258]])
tensor([[ 0.0000,  0.0000,  0.0000],
        [-0.6275,  0.0000,  0.0000],
        [-0.1785,  1.7177,  0.0000]])
'''

# 使用torch.diag取对角线元素, 使用torch.diag_embed()恢复维度
example = torch.randn(3,3)
print(example)
diag = torch.diag(example)
print(diag)
print(torch.diag_embed(diag))
'''
tensor([[-0.2535,  0.3514, -0.5557],
        [-0.3072,  0.0542,  0.8716],
        [-1.0501, -1.0417,  1.1511]])
tensor([-0.2535,  0.0542,  1.1511])
tensor([[-0.2535,  0.0000,  0.0000],
        [ 0.0000,  0.0542,  0.0000],
        [ 0.0000,  0.0000,  1.1511]])
'''

# torch.diagonal(input, offset, dim1, dim2), dim1和dim2表示要取对角线元素的矩阵维度, 默认为0和1
example = torch.randn(3,3)
print(example)
print(torch.diagonal(example, 0))
print(torch.diagonal(example, 1))
'''
tensor([[-0.7329, -1.5077,  0.5697],
        [-1.5408,  0.0761,  0.4745],
        [-0.2090,  0.0796,  0.5854]])
tensor([-0.7329,  0.0761,  0.5854])
tensor([-1.5077,  0.4745])
'''
# 取一个batch的对角线元素
matrix = torch.randn(2,3,3)
print(matrix)
torch.diagonal(matrix, dim1=-2, dim2=-1)
'''
tensor([[[ 0.2858, -1.5002, -0.7323],
         [ 0.1168,  0.5091,  0.1758],
         [ 0.8847, -1.2018,  0.6861]],

        [[ 0.8108, -1.1476,  1.3786],
         [ 0.1905, -1.8077, -1.0145],
         [-0.5510, -0.3776,  0.2339]]])
tensor([[ 0.2858,  0.5091,  0.6861],
        [ 0.8108, -1.8077,  0.2339]])
'''

a = torch.arange(1, 10).view(3,3)
print(a)
print(a.trace()) # 对角线元素之和, 矩阵的迹
'''
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
tensor(15)
'''

# (8): 切分
# torch.chunk(tensor, chunk数, 维度）
a=torch.tensor([[1,2],[3,4]])
torch.chunk(a,2,1)
'''
(tensor([[1],
         [3]]), tensor([[2],
         [4]]))
'''

a = torch.rand(4,8,6)
print(torch.split(a, 2, dim=0)) # 按照4这个维度拆分, 结果是个元组, 元素一和元素二都是2x8x6

# (9): 堆叠和拼接
# torch.cat和torch.stack的区别在于torch.cat沿着给定的维度拼接, 而torch.stack会新增一维

# torch.stack(tensors, dim=0)
v = torch.Tensor([1, 2, 3, 4]) 
w = torch.Tensor([1, 0, 2, 0])
torch.stack((v,w))
'''
tensor([[1., 2., 3., 4.],
        [1., 0., 2., 0.]])
'''
torch.stack([torch.tensor(0), torch.tensor(1)]) # tensor([0, 1]) 

list_of_tensors = [torch.Tensor(2, 3), torch.Tensor(2, 3), torch.Tensor(2, 3)]
t1 = torch.cat(list_of_tensors, dim=0)
t2 = torch.stack(list_of_tensors, dim=0)
print(t1.shape, t2.shape) # torch.Size([6, 3]) torch.Size([3, 2, 3])

In [ ]:
# 张量运算

# (1): 比较
# 判断两个张量相等
# int tensor
a = torch.arange(5)
b = torch.arange(5)
torch.equal(a, b) # True
# float tensor
c = torch.tensor([0.1, 0.2])
d = torch.tensor([0.1, 0.2])
torch.equal(c, d) # True
torch.allclose(c, d) # True
c = torch.tensor([0.100001, 0.20000001])
d = torch.tensor([0.1, 0.2])
torch.equal(c, d) # False
torch.allclose(c, d) # True

a = torch.arange(5)
b = torch.arange(5)
torch.eq(a, b) # tensor([True, True, True, True, True]); torch.ne(a, b)
x=torch.randn(2,4)
y=torch.randn(2,4)
print(x)
print(y)
print(torch.le(x, y))
print(torch.ge(x, y)) # x中元素大于(严格大于)y中对应元素, 大于则为True, 不大于则为False
'''
tensor([[ 0.9681, -1.2554, -0.3515, -0.0979],
        [ 0.3441, -0.2064, -1.5022,  0.5089]])
tensor([[-0.0652, -1.1615, -0.6938, -0.5722],
        [-0.1181,  0.4765,  0.6100,  1.5423]])
tensor([[False,  True, False, False],
        [False,  True,  True,  True]])
tensor([[ True, False,  True,  True],
        [ True, False, False, False]])
'''

# 最值
a = torch.randn(2,3)
print(a)
torch.max(a,0) # 按维度dim返回最大值, 并且返回索引
'''
tensor([[-0.0137,  0.3792, -2.1112],
        [ 1.2163,  2.3935, -2.0245]])
torch.return_types.max(
values=tensor([ 1.2163,  2.3935, -2.0245]),
indices=tensor([1, 1, 1]))
'''
torch.argmax()

# (2): 四则运算
a.mul_(2) # tensor([0, 2, 4, 6, 8])

# 向量乘法
# *: Element-wise乘法/Hadamard product
v = torch.Tensor([1, 2, 3, 4]) 
w = torch.Tensor([1, 0, 2, 0])
v * w # tensor([1., 0., 6., 0.])
# @: Scalar product
v @ w # tensor(7.)=1*1 + 2*0 + 3*2 + 4*0

# 矩阵向量乘法
m = torch.Tensor([[2, 5, 3, 7],[4, 2, 1, 9]])
v = torch.arange(1., 5)
print(m @ v) # tensor([49., 47.])
print(m[[0], :] @ v) # tensor([49.]) 

# 矩阵乘法
tensor1 = torch.tensor([[0,1,2], [2,4,1]])
tensor2 = torch.tensor([[0,1], [2,4], [1,2]])
torch.mm(tensor1, tensor2) # (m*n) * (n*p) -> (m*p); torch.matmul类似, 不过torch.mm针对二维矩阵, torch.matmul是高维, 当torch.mm用于大于二维时将报错; 还有@类似
'''
tensor([[ 4,  8],
        [ 9, 20]])
'''
torch.bmm(a,b) # 矩阵乘法, tensor a的size为(b,h,w), tensor b的size为(b,w,m), 两个tensor的维度必须为3
tensor1 = torch.tensor([[0,1,2], [2,4,1]])
tensor2 = torch.tensor([[0,1,2], [2,4,1]])
tensor1 * tensor2 # torch.mul类似
# 当a, b维度不一致时, 会自动填充到相同维度相点乘
'''
tensor([[ 0,  1,  4],
        [ 4, 16,  1]])
'''

v = torch.arange(1, 5)
v.pow(2) # tensor([ 1,  4,  9, 16])

# 求余
a = torch.FloatTensor([5, 6, 7, 4]) 
torch.fmod(a, 3) # tensor([2., 0., 1., 1.])
print(torch.fmod(torch.tensor([-3., -2, -1, 1, 2, 3]), 2)) # tensor([-1., -0., -1.,  1.,  0.,  1.])

a = tensor([ 0.7000, -1.2000,  0.0000,  2.3000])
torch.sign(a) # tensor([ 1., -1.,  0.,  1.])

torch.abs()

torch.sqrt()
# 计算两组数据之间的两两欧式距离
# X1 is of shape m*d.
X1 = torch.tensor([[0,1], [2,4], [1,2]])
X1 = torch.unsqueeze(X1, dim=1).expand(3, 2, 2)
# X2 is of shape n*d.
X2 = torch.tensor([[2,4], [1,2]])
X2 = torch.unsqueeze(X2, dim=0).expand(3, 2, 2)
# dist is of shape m*n, where dist[i][j] = sqrt(|X1[i, :] - X[j, :]|^2)
dist = torch.sqrt(torch.sum((X1 - X2) ** 2, dim=2))
# 或者
dist = torch.sqrt(torch.sum((X1[:,None,:] - X2) ** 2, dim=2)) # 利用broadcast机制

torch.exp()

# (3): 范数计算
torch.norm # 同np.linalg.norm
# 参数p是范数计算中的幂指数值 
# 参数keepdim是保持输出的维度, 默认是False; 当keepdim=False时, 输出比输入少一个维度(就是指定的dim求范数的维度); 而keepdim=True时, 输出与输入维度相同, 仅仅是输出在求范数的维度上元素个数变为1; 这也是为什么有时我们把参数中的dim称为缩减的维度，因为norm运算之后，此维度或者消失或者元素个数变为1
a = torch.ones(2,3)
a2 = torch.norm(a)      # 默认求2范数
a1 = torch.norm(a,p=1)  # 指定求1范数
print(a2) # tensor(2.4495)
print(a1) # tensor(6.)
# 求指定维度上的范数
a = torch.tensor([[1, 2, 3, 4], [1, 2, 3, 4]]).float()  # norm仅支持floatTensor
a0 = torch.norm(a,p=2,dim=0)    # 按0维度求2范数
a1 = torch.norm(a,p=2,dim=1)    # 按1维度求2范数
print(a0) # tensor([1.4142, 2.8284, 4.2426, 5.6569])
print(a1) # tensor([5.4772, 5.4772])
# keepdim
a = torch.rand(2,3,4) 
at = torch.norm(a,p=2,dim=1,keepdim=True)   # 保持维度
af = torch.norm(a,p=2,dim=1,keepdim=False)  # 不保持维度
print(at)
print(af)
'''
tensor([[[0.8443, 0.9360, 1.2900, 0.6349]],

        [[0.9302, 0.9422, 1.2238, 0.9469]]])
tensor([[0.8443, 0.9360, 1.2900, 0.6349],
        [0.9302, 0.9422, 1.2238, 0.9469]])
'''

# (4): 线性代数
# torch.linalg
torch.linalg.slogdet # 返回符号和方阵行列式绝对值的自然对数, 同torch.slogdet
A = torch.randn(3, 3)
print(torch.det(A))
print(torch.logdet(A))
print(torch.slogdet(A))
'''
tensor(0.3191)
tensor(-1.1421)
torch.return_types.slogdet(
sign=tensor(1.),
logabsdet=tensor(-1.1421))
'''

torch.inverse() # 并不是所有的矩阵都可逆, 对不可逆矩阵进行求逆会报错

# (5): Einstein
# torch.einsum(equation, *operands): 基于爱因斯坦求和规则
# 如矩阵乘法: torch.einsum("ij,jk->ik", A, B)  j是summation subscript, i和k是output subscripts
# '...v,...uv->...u'
# 标有相同下标的维度必须是可广播的, 也就是说, 它们的大小必须匹配或为1
tensor1 = torch.tensor([[0,1,2], [2,4,1]])
tensor2 = torch.tensor([[0,1], [2,4], [1,2]])
output = torch.einsum('ik, kj -> ij', tensor1, tensor2)
# 'ik, kj -> ij'语义解释如下: 
# 输入tensor1: 2维数组, 下标为ik
# 输入tensor2: 2维数组, 下标为kj
# 输出output: 2维数组, 下标为ij
# 隐含语义: 输入a,b下标中相同的k, 是求和的下标
print(output)
'''
tensor([[ 4,  8],
        [ 9, 20]])
'''
A = torch.randn(3, 4)
torch.einsum('ij->', A) # tensor(-3.2456), 求和
torch.einsum('ij->j', A) # tensor([-3.7191, -0.3284,  1.8927,  3.9906]), 列求和

# (6): 傅立叶变换
# torch.fft(input, signal_ndim, normalized=False) 离散傅立叶变换
# input的维度至少是signal_ndim + 1
# signal_ndim: 每个信号的维数, 只能取1, 2, 3, 分别支持1D, 2D和3D complex-to-complex变换
# 返回张量包含complex-to-complex傅立叶变换结果
a = torch.FloatTensor([[5, 6], [7, 4]]) 
torch.fft(a, 1)
'''
tensor([[12., 10.],
        [-2.,  2.]])
'''
torch.ifft()

In [ ]:
# CPU/GPU相关

# cuda相关
torch.version.cuda # 10.2
torch.cuda.get_device_name(0) # NVIDIA GeForce RTX 2080 Ti
torch.cuda.empty_cache() # 清除显存
torch.backends.cudnn.version() # 7605
torch.backends.cudnn.deterministic 
# 打开CUDNN deterministic设置, 会降低训练速度; 且从checkpoints重新开始时会出现意外的结果
torch.backends.cudnn.benchmark 
# 设置为True, 就可以大大提升卷积神经网络的运行速度, 可在网络训练开始前设置

# torch.set_num_threads(n_threads)
	# 参数的意义: Number of threads used for parallelizing CPU operations 

# 用 del 及时删除不用的中间变量, 节约 GPU 存储
# 使用 inplace 操作可节约 GPU 存储
	# x = torch.nn.functional.relu(x, inplace=True)
# 使用半精度浮点数 half() 会有一定的速度提升, 具体效率依赖于 GPU 型号
	# 需要小心数值精度过低带来的稳定性问题

# 减少 CPU 和 GPU 之间的数据传输
	# 如果你想知道一个 epoch 中每个 mini-batch 的 loss 和准确率
	# 先将它们累积在 GPU 中等一个 epoch 结束之后一起传输回 CPU 会比每个 mini-batch 都进行一次 GPU 到 CPU 的传输更快

# 设备
a.device # device(type='cpu')
device = torch.device("cuda")
a = a.to(device)
a.device # device(type='cuda', index=0)
device = torch.device("cuda:1")
a = a.to(device)
a.device # device(type='cuda', index=1)
# 使用单个gpu
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # device=-1指cpu

# tensor = tensor.cuda()

In [ ]:
# 存储和下载

# 存储和下载整个模型
resnet = None
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')
# torch.load(map_location=torch.device(f'cuda:{str(gpu)}'))
	# map_location指明此时的device环境

# 存储和下载模型参数
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))
# 函数load_state_dict有参数strict, 默认strict=True; 判断参数拷贝过程中是否有unexpected_keys或者missing_keys, 如果有就报错, 代码不能继续执行; 如果strict=False, 则会忽略这些细节
# resnet.state_dict()返回的是一个OrderDict, 存储了网络结构的名字和对应的参数
# 同理, optimizer.load_state_dict(optimizer.state_dict())